In [25]:
import pandas as pd
import numpy as np
import tensorflow as tf
import autokeras as ak

In [30]:
from konlpy.tag import Mecab
mecab = Mecab()
train = pd.read_csv("1.csv")
test = pd.read_csv("2.csv")
train['title'] = train['title'].map(lambda x: ' '.join(mecab.morphs(x)))
test['title'] = test['title'].map(lambda x: ' '.join(mecab.morphs(x)))
print(train)
print(test)

                                                 title      level
0                   안면 인식 CCTV 도입 으로 중국식 감시 사회 가 도래 한다   전혀 사실 아님
1    영업 제 한 업종 소상 공인 은 2019 년 대비 2020 년 총 매출 이 감소 해...     대체로 사실
2                      코로나 19 로 인해 우리 나라 사망 률 이 급증 했 다  대체로 사실 아님
3    문 대통령 이 본인 이 먼저 코로나 19 백신 을 맞 을 수 있 다고 TV 에 나와...     절반의 사실
4                      아스트라 제 네 카 백신 은 안전 성 에 문제 가 있 다  대체로 사실 아님
..                                                 ...        ...
255             백신 ' 공급 시기 ' 합의 했어도 계약서 에 안 쓰 면 위법 이 다       판단유보
256   세계 최초 코로나 19 백신 접종 자 의 사진 이 10 월 에 이미 보도 된 적 있 다   전혀 사실 아님
257  해외 제약사 들 이 광범위 한 면책 을 요구 하 는 것 은 국제 적 으로 거의 공통...     대체로 사실
258                           아스트라 제 네 카 백신 은 중국 산 이 다  대체로 사실 아님
259       다른 나라 는 아스트라 제 네 카 대신 화이자 , 모더 나 백신 을 선택 했 다   전혀 사실 아님

[260 rows x 2 columns]
                                                 title  level
0                [ 속보 ] 화성시 팔탄면 율암리 현대 한식 부페 방문자 검사 요청      0
1    [ 속보 ] 화성 시청 현대 한식 부페 ( 팔탄면 서해 로 1121 번 길 11 )...      

In [32]:
# change_value_dict = {'판단유보':0,'전혀 사실 아님' : 0,'대체로 사실 아님' : 0.25,'절반의 사실' : 0.5,'대체로 사실' : 0.75, '사실' : 1}
# train = train.replace({'level' : change_value_dict})
print(train)
train['title'] = train['title'].map(lambda x: ' '.join(mecab.morphs(x)))
test['title'] = test['title'].map(lambda x: ' '.join(mecab.morphs(x)))
X = train['title'].values
Y =  train['level'].values

                                                 title      level
0                   안면 인식 CCTV 도입 으로 중국식 감시 사회 가 도래 한다   전혀 사실 아님
1    영업 제 한 업종 소상 공인 은 2019 년 대비 2020 년 총 매출 이 감소 해...     대체로 사실
2                      코로나 19 로 인해 우리 나라 사망 률 이 급증 했 다  대체로 사실 아님
3    문 대통령 이 본인 이 먼저 코로나 19 백신 을 맞 을 수 있 다고 TV 에 나와...     절반의 사실
4                      아스트라 제 네 카 백신 은 안전 성 에 문제 가 있 다  대체로 사실 아님
..                                                 ...        ...
255             백신 ' 공급 시기 ' 합의 했어도 계약서 에 안 쓰 면 위법 이 다       판단유보
256   세계 최초 코로나 19 백신 접종 자 의 사진 이 10 월 에 이미 보도 된 적 있 다   전혀 사실 아님
257  해외 제약사 들 이 광범위 한 면책 을 요구 하 는 것 은 국제 적 으로 거의 공통...     대체로 사실
258                           아스트라 제 네 카 백신 은 중국 산 이 다  대체로 사실 아님
259       다른 나라 는 아스트라 제 네 카 대신 화이자 , 모더 나 백신 을 선택 했 다   전혀 사실 아님

[260 rows x 2 columns]


In [34]:

input_node = ak.TextInput()
output_node = ak.TextToIntSequence()(input_node)
output_node = ak.Embedding()(output_node)
output_node = ak.ConvBlock(separable=True)(output_node)
output_node = ak.ClassificationHead()(output_node)
clf = ak.AutoModel(
    inputs=input_node,
    outputs=output_node,
    overwrite=True,
    max_trials=5
    )
clf.fit(X, Y, epochs=5)
model = clf.export_model()

Trial 5 Complete [00h 00m 02s]
val_loss: 1.423948884010315

Best val_loss So Far: 1.4022222757339478
Total elapsed time: 00h 00m 12s
INFO:tensorflow:Oracle triggered exit
Epoch 1/5
9/9 [==============================] - 1s 12ms/step - loss: 1.7853 - accuracy: 0.2651
Epoch 2/5
9/9 [==============================] - 0s 12ms/step - loss: 1.7303 - accuracy: 0.4585
Epoch 3/5
9/9 [==============================] - 0s 12ms/step - loss: 1.6016 - accuracy: 0.4585
Epoch 4/5
9/9 [==============================] - 0s 12ms/step - loss: 1.4367 - accuracy: 0.4585
Epoch 5/5
9/9 [==============================] - 0s 13ms/step - loss: 1.4111 - accuracy: 0.4585
INFO:tensorflow:Assets written to: ./auto_model/best_model/assets


In [35]:
sample_submission = pd.read_csv("sample_submission.csv")
pred_test = model.predict(test['title'].values)

In [36]:
test['level']=pred_test
test

,title,level
0,[ 속보 ] 화성시 팔탄면 율암리 현대 한식 부페 방문자 검사 요청,0.042380
1,[ 속보 ] 화성 시청 현대 한식 부페 ( 팔탄면 서해 로 1121 번 길 11 )...,0.046034
2,"세계 은행 , 내달 까지 40 개 개도국 에 백신 자금 20 억 달러 지원",0.043883
3,美 일부 주 J & J 백신 중단 … 유럽 의 약 품청 혈전 사례 검토,0.044208
4,군내 코로나 19 확진 자 2 명 추가 … 누적 673 명,0.042685
...,...,...
495,"BTJ 열방 센터 , 코로나 19 방역 수칙 위반 등 관련 사과문 발표",0.042949
496,"대구시 교육청 , 초 , 중 , 고 영재 학급 영재교육 대상자 선발 전형 실시",0.042779
497,"경북 교육청 , ' 교육 복지 우선 지원 사업 ' 연수 회 개최",0.042537
498,"이강덕 포항시장 , 김태성 해병대 사령관 내정자 에게 명예 시민증 수여",0.042396


In [37]:
test.to_csv("sample_submission.csv",header=False,index=False,encoding='utf-8')